In [ ]:
# !pip install d2l==1.0.3
# !pip install matplotlib_inline

In [ ]:
import torch
import torch.nn as nn
import shutil
import os
from torch.utils.data import Dataset, DataLoader
import unittest
from d2l import torch as d2l
import logging
import requests
import pickle
from tqdm import tqdm
import numpy as np
import shutil

### Download the Spanish-English dataset

In [ ]:
# Google Collab
!mkdir data
!curl https://www.manythings.org/anki/spa-eng.zip -o data/spa-eng.zip
!unzip data/spa-eng.zip -d data/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5286k  100 5286k    0     0  2587k      0  0:00:02  0:00:02 --:--:-- 2598k
Archive:  data/spa-eng.zip
  inflating: data/_about.txt         
  inflating: data/spa.txt            


In [ ]:
#Windows
# !mkdir data
# !curl https://www.manythings.org/anki/spa-eng.zip -o data/spa-eng.zip
# !tar -xf data/spa-eng.zip -C data

### Setting up logger

In [ ]:
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

### Dataset
###  Process the english spanish translation
### link: https://www.manythings.org/anki/

In [ ]:
class SpanishDataset(Dataset):

    def setup_logger(self, level = logging.DEBUG):
        self.logger = logging.getLogger()
        self.logger.setLevel(level)

    def log(self, message:str, level: str = 'debug'):
        if level == 'debug':
            self.logger.debug(message)
        if level == 'info':
            self.logger.info(message)
        if level == 'warning':
            self.logger.warning(message)
        if level == 'error':
            self.logger.error(message)


    def __init__(self, debug = False, num_steps = 10):
        super().__init__()

        self.setup_logger()

        self.DATASET_PATH = './data/spa.txt'
        assert os.path.exists(self.DATASET_PATH), 'English spanish dataset is not found'
        self.num_steps = num_steps

        self.source = []
        self.target = []

        self.log('start building the dataset')

        with open(self.DATASET_PATH, 'r') as file:
            for idx, line in enumerate(file.readlines()):
                processed = self._preprocess(line)
                source_tokens, target_tokens = self._tokenize(processed)
                self.source.append(source_tokens)
                self.target.append(target_tokens)

        self.log(f'done tokenizing source and target, source len = {len(self.source)}, target len = {len(self.target)}', 'info')

        (self.source_array, self.target_array, self.valid_len, self.label_target_array), self.source_vocab, self.target_vocab = \
            self._build_arrays(self.source, self.target)

        #add tgt_pad (target pad) for masking in training phase
        self.tgt_pad = self.target_vocab['<pad>']

        shape2d = lambda a: f'({len(a)},{len(a[0])})'
        self.log(f'done building source and target arrays', 'info')
        self.log(f'source array shape {shape2d(self.source_array)}')
        self.log(f'source vocab len = {len(self.source_vocab)}')
        self.log(f'valid_len shape  = {self.valid_len.shape}')
        self.log(f'target array shape = {shape2d(self.target_array)}')
        self.log(f'target vocab len = {len(self.target_vocab)}')

    def _preprocess(self, text):
        # from D2L processing step in chapter 10
        # Replace non-breaking space with space
        text = text.replace('\u202f', ' ').replace('\xa0', ' ')
        # Insert space between words and punctuation marks
        no_space = lambda char, prev_char: char in ',.!?' and prev_char != ' '
        out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
            for i, char in enumerate(text.lower())]
        return ''.join(out)

    def _tokenize(self, text):
        # Tokenization method in D2L processing step in chapter 10
        if len(text.split('\t')[:-1]) == 2:
            part = text.split('\t')[:-1]
            src = [token for token in f'{part[0]} <eos>'.split(' ') if token]
            tgt = [token for token in f'{part[1]} <eos>'.split(' ') if token]
            return src, tgt
        else:
            return '',''
    def _build_arrays(self, source, target):
        '''
        @params:
            source_raw: list[list[string]], source sequence, eg: [['a', 'b', '<eos>'], ...]
            target_rwa: list[list[string]], target sequence
        @return
            (
                source_array: list[list[int]]
                target_array_with_bos: list[list[int]]
                valid_len: list[int]
                target_array_with_eos: list[list[int]]
            ),
            source_vocab: Vocab
            target_vocab: Vocab
        '''
        #pad with <pad> token if sequence len < time step, else truncate
        #NOTE: in the book, they just truncated without adding <eos> at the end,
        # I don't think that is correct
        pad_or_truncate = lambda sentence, numstep: \
            sentence[:numstep - 1] + ['<eos>'] if len(sentence) > numstep \
                else sentence + ['<pad>'] * (numstep - len(sentence))

        def _build_array(sequence, is_target = False):
            '''
            @params:
                sentence: string
                is_target: boolean, if sentence is target, append <bos> to beginning of sentence
            @return
                array: list[str]
                vocab: Vocab object
            '''
            new_sequence = [ ]
            for sentence in sequence:
                sentence = pad_or_truncate(sentence, self.num_steps)
                if is_target:
                    sentence = ['<bos>'] + sentence

                new_sequence.append(sentence)

            vocab = d2l.Vocab(new_sequence, min_freq = 2)

            #calculate valid_len for training later
            array = torch.tensor([vocab[sentence] for sentence in new_sequence])
            valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
            return array,vocab,valid_len

        source_array, source_vocab, valid_len = _build_array(source)
        target_array, target_vocab, _ = _build_array(target, is_target= True)

        return (source_array, target_array[:,:-1], valid_len, target_array[:,1:]), source_vocab, target_vocab

    def __len__(self):
        '''
        @return
            int: length of the english - spanish pairs
        '''
        return len(self.source_array)

    def __getitem__(self,idx):
        '''
        @params:
            idx: int, datapoint index
        @return
            source_array, target_array, valid_len, label_target_array
        '''
        return (self.source_array[idx], self.target_array[idx], self.valid_len[idx], self.label_target_array[idx])

    def get_dataloader(self, **kwargs):
        return DataLoader(self, **kwargs)


In [ ]:
# dataset = SpanishDataset()
# source = dataset.source
# target = dataset.target# print(dataset.source_array[0])

# print(dataset.source_vocab.to_tokens(dataset.source_array[500].tolist()))

# train_dataloader = dataset.get_dataloader()
# sample = next(iter(train_dataloader))

# for data in sample:
#     print(f'shape = {data.shape},\t data = {data[0]}')

tensor([3825,   84,  201,  202,  202,  202,  202,  202,  202,  202])
["i'm", 'warm', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


### Encoder and Decoder Architecture

In [ ]:
class Encoder(nn.Module):

    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(input_size = embed_size, num_layers = num_layers, hidden_size = num_hiddens, dropout = dropout)
        #custom initialization

    def forward(self, x, *args):
        #why x.t(), still confused in the book
        emb = self.embedding(x.t())
        output, state = self.rnn(emb)
        return output, state

class Decoder(nn.Module):

    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout = 0):
        #note that the vocab size in decoder is target language vocab size, not source language vocab size
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(input_size = embed_size + num_hiddens, num_layers = num_layers, hidden_size = num_hiddens, dropout = dropout)
        self.dense = nn.LazyLinear(vocab_size)
        #custom init module

    def init_state(self, enc_output, *args):
        '''
        use encoder's output to initialize state
        @params:
            encoder_output
        @return:
            decoder_input
        '''
        return enc_output

    def forward(self, x, state):
        emb = self.embedding(x.t())

        enc_output, enc_state = state
        #context variable
        context = enc_output[-1]
        context = context.repeat(emb.shape[0], 1, 1)
        emb_and_context = torch.cat((emb, context), -1)
        dec_output, dec_state = self.rnn(emb_and_context, enc_state)

        #pass to dense layer and swap back (batch_size, num_steps)
        y_pred = self.dense(dec_output).swapaxes(0,1)
        # print(x.shape)
        # print('embedding shape = ', emb.shape)
        # print('context shape = ', context.shape)
        # print('emb and context shape = ', emb_and_context.shape)
        # print('decoder output shape = ', output.shape)
        # print('decoder state shape = ', dec_state.shape)
        return y_pred, (dec_output, dec_state)

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec, *args):
        enc_outputs, enc_state = self.encoder(x_enc, *args)
        dec_init_state = self.decoder.init_state((enc_outputs, enc_state), *args)
        #only returns decoder output, decode state is not used for final pred
        return self.decoder(x_dec, dec_init_state)[0]


# Test Encoder

In [ ]:
# Test with sample
vocab_size, embed_size, num_layers, num_hiddens = 1000, 8, 2, 16
batch_size, num_steps = 32, 10

x = torch.randint(0,vocab_size,(batch_size, num_steps))

encoder = Encoder(vocab_size, embed_size, num_hiddens, num_layers )

output, state = encoder(x)

print('output shape = ',output.shape, ' dtype = ', output.dtype)
print('state shape = ', state.shape, ' dtype = ', state.dtype)

output shape =  torch.Size([10, 32, 16])  dtype =  torch.float32
state shape =  torch.Size([2, 32, 16])  dtype =  torch.float32


# Test Decoder

In [ ]:
# Test with sample
enc_vocab_size, embed_size, num_layers, num_hiddens = 1000, 8, 2, 16
dec_vocab_size = 2000
batch_size, num_steps = 32, 10

x = torch.randint(0,enc_vocab_size,(batch_size, num_steps))

encoder = Encoder(enc_vocab_size, embed_size, num_hiddens, num_layers )

enc_output, enc_state = encoder(x)

print('enc output shape = ',enc_output.shape, ' dtype = ', enc_output.dtype)
print('enc state shape = ', enc_state.shape, ' dtype = ', enc_state.dtype)

decoder = Decoder(dec_vocab_size, embed_size, num_hiddens, num_layers)
decoder_state = decoder.init_state((enc_output, enc_state))

y, (dec_output, dec_state) = decoder(x, decoder_state)

print('y_pred shape = ', y.shape, 'dtype = ', dec_output.dtype)
print('dec state shape  = ', dec_state.shape, 'dtype = ', dec_state.dtype)


enc output shape =  torch.Size([10, 32, 16])  dtype =  torch.float32
enc state shape =  torch.Size([2, 32, 16])  dtype =  torch.float32
y_pred shape =  torch.Size([32, 10, 2000]) dtype =  torch.float32
dec state shape  =  torch.Size([2, 32, 16]) dtype =  torch.float32


/Users/jamesnguyen/anaconda3/envs/torch/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


# Test Encoder Decoder

In [ ]:
#Test seq 2 seq
# Test with sample
vocab_size_source, embed_size, num_layers, num_hiddens = 1000, 8, 2, 16
vocab_size_target = 2000
batch_size, num_steps = 32, 10

#this represents the source_array
x_enc = torch.randint(0,vocab_size_source,(batch_size, num_steps))
#this represents the target array with BOS
x_dec = torch.randint(0,vocab_size_target,(batch_size, num_steps))

encoder = Encoder(vocab_size_source, embed_size, num_hiddens, num_layers)
decoder = Decoder(vocab_size_target, embed_size, num_hiddens, num_layers)

seq2seq = Seq2Seq(encoder, decoder)

output = seq2seq(x_enc, x_dec)
print('seq2seq output shape = ', output.shape)


seq2seq output shape =  torch.Size([32, 10, 2000])


### Perplexity

Perplexity is PPL = e^(CrossEntropyLoss(y_pred, y))

### Training procedure

In [ ]:

class Trainer():
  def __init__(self, model, dataset):
    self.model = model
    self.dataset = dataset
    self._run()
    if torch.cuda.is_available():
      self.device = 'cuda'
    else:
      self.device = 'cpu'

  def _has_gpu(self):
    return self.device == 'cuda'

  def prepare_batch(self, batch):
    if self._has_gpu():
      batch = [a.to(self.device) for a in batch]
    return batch

  def _run(self):
    pass

In [ ]:
#Build the dataset
dataset = SpanishDataset()

#Extract some paramaters
source_vocab_size = len(dataset.source_vocab)
print(f'source vocab size = {source_vocab_size}')
target_vocab_size = len(dataset.target_vocab)
print(f'target vocab size = {target_vocab_size}')
embed_size = 100
num_hiddens = 256
num_layers = 2
dropout = 0.2
batch_size = 32


DEBUG - start building the dataset
INFO - done tokenizing source and target, source len = 141370, target len = 141370
INFO - done building source and target arrays
DEBUG - source array shape (141370,10)
DEBUG - source vocab len = 9538
DEBUG - valid_len shape  = torch.Size([141370])
DEBUG - target array shape = (141370,10)
DEBUG - target vocab len = 16679


source vocab size = 9538
target vocab size = 16679


In [ ]:
class PerplexityLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.loss_fn = nn.CrossEntropyLoss()

  def forward(self, y_pred, y, tgt_pad):
    y_pred = y_pred.permute(0,2,1)
    l = self.loss_fn(y_pred, y)
    mask = (y.reshape(-1) != tgt_pad).type(torch.float32)
    return torch.exp((l * mask).sum() / mask.sum())

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Get dataloader for
train_dataloader = dataset.get_dataloader(batch_size = batch_size, shuffle = True)
print('train dataloader len = ',len(train_dataloader))

encoder = Encoder(source_vocab_size, embed_size, num_layers, num_hiddens)
decoder = Decoder(target_vocab_size, embed_size, num_layers, num_hiddens)

seq2seq = Seq2Seq(encoder, decoder)

seq2seq.to(device)

ppl_loss = PerplexityLoss()
optim = torch.optim.Adam(seq2seq.parameters(), lr = 0.01)

history = []

epochs = 20

for e in range(epochs):
  loop = tqdm(train_dataloader)
  running_loss =0

  for data in loop:
    src_array, tgt_array_bos, valid_len, tgt_array_eos = data
    src_array = src_array.to(device)
    tgt_array_bos = tgt_array_bos.to(device)
    tgt_array_eos = tgt_array_eos.to(device)
    valid_len = valid_len.to(device)
    # print('src array shape = ', src_array.shape)
    # print(f'tgt array bos shape {tgt_array_bos.shape}' )
    # print(f'tgt array eos shape {tgt_array_eos.shape}')
    # print(f'shape {valid_len.shape}')

    y_pred = seq2seq(src_array, tgt_array_bos)
    # print('y pred ', y_pred[0])
    # print('y pred GRAD', y_pred.grad)
    # print('target ', tgt_array_eos[0])

    optim.zero_grad()
    loss = ppl_loss(y_pred, tgt_array_eos, dataset.tgt_pad)
    loss.backward()
    step = optim.step()
    running_loss += loss.item()
    loop.set_description(f'batch ppl = {loss.item()}')

  print(f'epoch = {e}, ppl = {running_loss}')
  history.append({'loss': running_loss})

train dataloader len =  4418


batch ppl = 12754.9326171875:   0%|          | 19/4418 [00:12<46:45,  1.57it/s]


KeyboardInterrupt: 

In [ ]:
if os.path.exists('output') == False:
    os.mkdir('output')
MODEL_PATH = './output/seq2seq.h5'
torch.save(seq2seq, MODEL_PATH)

# Evaluating seq2seq model

In [ ]:
MODEL_PATH = './output/seq2seq.h5'
model = torch.load(MODEL_PATH, map_location = torch.device('cpu'))
print(type(model))
dataset = SpanishDataset()

DEBUG - start building the dataset


<class '__main__.Seq2Seq'>


INFO - done tokenizing source and target, source len = 141370, target len = 141370
INFO - done building source and target arrays
DEBUG - source array shape (141370,10)
DEBUG - source vocab len = 9538
DEBUG - valid_len shape  = torch.Size([141370])
DEBUG - target array shape = (141370,10)
DEBUG - target vocab len = 16679


In [ ]:
index = []
for i in range(10):
  index.append(np.random.choice(len(dataset)))

batch = []
src = []
tgt = []
valid_len = []
tgt_eos = []

for i in index:
  data = dataset[i]
  src.append(data[0])
  tgt.append(data[1])
  valid_len.append(data[2])
  tgt_eos.append(data[3])

batch = [torch.stack(a) for a in (src,tgt,valid_len,tgt_eos)]
print(batch)

[tensor([[4370, 8244, 8641, 9349,  960,  225, 8623, 2702, 8488,  201],
        [4367, 2725, 4093, 8491, 7687, 4377, 9287, 4683,  235,  201],
        [4367, 6773, 8491, 1191, 4906, 5682,   84,  201,  202,  202],
        [4101, 7405,  206, 4994,  353, 8623, 4219, 8890,   84,  201],
        [3108,  206, 1679, 1461, 8904, 4680,   84,  201,  202,  202],
        [4367, 2057, 7610,   84,  201,  202,  202,  202,  202,  202],
        [9231, 3838, 8623, 8491, 5530,   84,  201,  202,  202,  202],
        [8010, 9303, 9512,  661,   84,  201,  202,  202,  202,  202],
        [4367, 1461, 1874, 9389, 9512, 4386, 9512, 9177, 5293,  201],
        [8491,  203,  203, 8488, 8641, 9178, 8623, 1410, 9196,  201]]), tensor([[  189,  6750, 13902,  4547, 12630, 14965, 11894,  7909,  6567,    78],
        [  189, 10625, 14765,  9021,  9964,  8249,  4547, 12782, 13845, 15199],
        [  189,  1132,  9231,  5820,  9257,    78,   190,   191,   191,   191],
        [  189, 16631,  6343, 15375,  2787,  5352,   196,

In [ ]:
def predict(seq2seq, batch, device, num_steps):
  batch = [a.to(device) for a in batch]
  src, tgt, valid_len, _ = batch

  enc_outputs, enc_state = seq2seq.encoder(src, valid_len)
  print('enc outputs ', enc_output.shape)
  print('enc state  ', enc_state.shape)
  dec_outputs, dec_state = seq2seq.decoder.init_state((enc_outputs, enc_state), valid_len)

  outputs, attention_weights = [tgt[:,(0)].unsqueeze(1), ], []
  # print(outputs)
  # print(outputs)
  for _ in range(num_steps):
    print(outputs[-1])
    print(outputs[-1].shape)
    Y, dec_state = seq2seq.decoder(outputs[-1], dec_state)
    break

predict(model, batch, 'cpu', 10)

enc outputs  torch.Size([10, 32, 16])
enc state   torch.Size([2, 10, 256])
[tensor([[189],
        [189],
        [189],
        [189],
        [189],
        [189],
        [189],
        [189],
        [189],
        [189]])]

tensor([[189],
        [189],
        [189],
        [189],
        [189],
        [189],
        [189],
        [189],
        [189],
        [189]])
torch.Size([10, 1])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 10 but got size 1 for tensor number 1 in the list.

### Unit Tests

In [ ]:
# class SpanishDatasetTest(unittest.TestCase):

#     def test_upper(self):
#         self.assertEqual('foo'.upper(), 'FOO')

#     def test_isupper(self):
#         self.assertTrue('FOO'.isupper())
#         self.assertFalse('Foo'.isupper())

#     def test_split(self):
#         s = 'hello world'
#         self.assertEqual(s.split(), ['hello', 'world'])

#     def test4(self):
#         self.assertEqual('foo', 'foo1')

# unittest.main(argv=[''], exit=False)